<h1 style="text-align: center; font-size: 24pt">ICR : Mini-Projet</h1>
<h2 style="text-align: center; font-size: 18pt">Logging</h2>
<h2 style="text-align: center; font-size: 18pt">Loïc Piccot - 21.05.2025</h2>

---

# Paramètres Argon2

- **Nombre de threads** : Il est recommandé d'en utiliser le maximum possible pour obliger l'attaquant à utiliser le même nombre, sous peine d'être serieusement ralenti (accélère aussi la connexion)
- **Taille mémoire** : La taille nécessaire à stocker les résultats intermédiaires d'Argon2. Ici le calcul est effectué en interne du device utilisateur et non sur le serveur. Le nombre de requettes de connexions simultanées n'est donc pas un facteur limitant. La taille mémoire dépend donc uniquement de la place qu'il est acceptable d'utiliser sur le device utilisateur.
- **Temps de connexion** : Il faut adapter le paramètre de complexité d'Argon2 pour s'approcher du temps de connexion souhaitable. Plus ce temps est élevé, plus la sécurité est renforcée (au détriment d'une expérience utilisateur légèrement dégradée). Etant donné qu'il s'agit d'une application d'échange de message potentiellement sensibles, on va forcer **2s** de temps de connexion.


---

# Fonctionnement des protocoles liés au logging
Cette section considère un canal client/serveur sécurisé par TLS.

## Enregistrement
- L'utilisateur génère aléatoirement un sel
- L'utilisateur hash son mot de passe avec **Argon2** : $\tau = argon2(pwd, salt)$
- L'utilisateur envoie ses informations au serveur (qui vérifie si le nom existe déjà): $user_{\text{infos}} = (name, \tau, salt)$

## Identification
- L'utilisateur envoie une requette au serveur avec son username
- Le serveur répond avec le sel associé
- L'utilisateur recalcul $\tau' = argon2(pwd, salt)$
- L'utilisateur transmets $(username, \tau')$ au serveur
- Le serveur vérifie et si le $\tau'$ reçu correspond bien au $\tau$ stocké lors de l'enregistrement.

## Changement de mot de passe
- L'utilisateur effectue la procédure d'identification
- L'utilisateur effectue la procédure d'enregistrement
- Le serveur vérifie si le nom existe et si l'utilisateur correspondant est bien connecté
- Le serveur remplace les anciennes valeurs $(\tau, salt)$ par celles reçues

---

# Implémentation
*Pour l'implémentation, on appelera le hash du mot de passe ($\tau$) : `pwd_verifier` puisqu'il sert à vérifier le mot de passe de l'utilisateur lors de sa connexion.*

## Includes

In [6]:
from __future__ import annotations     # Postponed evaluation of annotations
from nacl import pwhash, secret, utils
from lphelpers import tracer

## KdfParams
Cette classe centralise tout les choix cryptographique pour la partie enregistrement et logging du projet. 

In [11]:
class KdfParams :
    def __init__(self, keysize=secret.SecretBox.KEY_SIZE, ops=pwhash.argon2id.OPSLIMIT_MODERATE, mem=pwhash.argon2id.MEMLIMIT_MODERATE) : 
        self.keysize = keysize
        self.ops = ops
        self.mem = mem

## UserInfos
Cette classe est un utilitaire pour grouper les informations utilisateurs enregistrés dans le serveur.

In [8]:
class UserInfos :
    def __init__(self, name:str, pwd_verifier:bytes, salt:bytes):
        self.name = name
        self.pwd_verifier = pwd_verifier
        self.salt = salt
        self.isconnected:bool = False

    def __str__(self):
        return f"{self.name}"

## Server
Cette classe permet d'instancier une entité serveur qui propose les méthodes suivantes : 
| Méthode | Fonction |
|---|---|
`register`| Permet à un utilisateur de s'enregister avec ses `UserInfos`
`getmysalt`| Permet à un utilisateur de récupérer son `salt` avant de se login
`login`| Permet à un utilisateur d'envoyer son `pwd_verifier` pour confirmer son identité et se connecter au serveur
`logout`| Permet à un utilisateur de se déconnecter
`update`| Permet à un utilisateur de mettre à jour son `pwd_verifier` en spécifiant le nouveau `salt` utilisé
`remove`| Supprimer un utilisateur
`show_registered_users`| Montrer la liste des utilisateurs enregistrés (seulement pour le débug)

In [13]:
class Server : 
    def __init__(self, name:str='Server', tr:tracer = tracer(trace_level='DEBUG')):
        self.name:str = name
        self.users:list[UserInfos]  = []

        self.tr:tracer = tr     # Tracer to handle general verbosity of the Server
                                # 4 possible levels : ERROR, WARNING, INFO, DEBUG

    def register(self, new_user: UserInfos) -> bool: 
        if all(user.name != new_user.name  for user in self.users):
            self.users.append(new_user)
            self.tr.info(f"[{self}]: New user {new_user.name} was added!")
            return True
        self.tr.error(f"[{self}]: User {new_user.name} already exists!")
        return False

    def getmysalt(self, requester_name: str) -> bytes :
        for user in self.users :
            if user.name == requester_name :
                return user.salt
        self.tr.error(f"[{self}]: No user is registered as : {requester_name}")
        return None
    
    def login(self, requester_name:str, requester_pwd_verifier:bytes) -> bool :
        for user in self.users :
            if user.name == requester_name :
                if user.pwd_verifier == requester_pwd_verifier :
                    user.isconnected = True
                    self.tr.info(f'[{self}]: User {requester_name} is now connected!')
                    return True
                else : 
                    self.tr.error(f'[{self}]: Wrong password!')
                    return False
        self.tr.error(f"[{self}]: No user is registered as : {requester_name}")
        return False
    
    def logout(self, requester_name) -> bool:
        for user in self.users:
            if user.name == requester_name:
                user.isconnected = False
                self.tr.info(f"[{self}]: {requester_name} has been logged out.")
                return True
        self.tr.error(f"[{self}]: No user is registered as: {requester_name}")
        return False

    def update(self, requester:UserInfos) -> bool :
        for user in self.users :
            if user.name == requester.name :
                if not user.isconnected :
                    self.tr.warn(f'[{self}]: Please login before!')
                    return False
                user.pwd_verifier = requester.pwd_verifier
                user.salt = requester.salt
                self.tr.info(f"[{self}]: Password updated for {user.name}")
                return True
        self.tr.error(f"[{self}]: No user is register as : {requester.name}")
        return False
    
    def remove(self, requester_name):
        self.users = [user for user in self.users if user.name != requester_name]
        self.tr.info(f"[{self}]: User {requester_name} was successfully removed!")

    def show_registered_users(self):
        for user in self.users :
            self.tr.info(user.name)

    def __str__(self):
        return f"{self.name}"

## Client
Cette classe permet d'instancier une entité client qui propose les méthodes suivantes : 
| Méthode | Fonction |
|---|---|
`gen_public_infos`| Permet à un utilisateur de générer ses `UserInfos` en hachant son mot de passe
`register_on`| Permet à un utilisateur de s'enregistrer auprès d'un `Server` en fournissant ses `UserInfos`
`login_on`| Permet à un utilisateur de s'enregistrer auprès d'un `Server`
`logout_from`| Permet à un utilisateur de se déconnecter d'un `Server` 
`change_password_on`| Permet à un utilisateur de mettre à jour son `pwd_verifier` en spécifiant le nouveau `salt` utilisé

In [12]:
class Client : 
    def __init__(self, name : str, password : bytes, tr:tracer = tracer(trace_level='DEBUG')):
        self.name = name
        self.password = password
        self.kdf = pwhash.argon2id.kdf
        self.kdf_params = KdfParams(keysize=secret.SecretBox.KEY_SIZE, ops=pwhash.argon2id.OPSLIMIT_SENSITIVE, mem=pwhash.argon2id.MEMLIMIT_SENSITIVE)

        self.tr:tracer = tr     # Tracer to handle general verbosity of the User
                                # 4 possible levels : ERROR, WARNING, INFO, DEBUG
    
    def gen_public_infos(self) -> UserInfos:
        self.tr.debug(f'[{self.name}]: Drawing a random salt...')
        salt = utils.random(pwhash.argon2i.SALTBYTES)                                       # Draw a random salt 
        self.tr.debug(f'[{self.name}]: Generating public informations...')
        kp = self.kdf_params                                                                # Get kdf parameters
        pwd_verifier = self.kdf(kp.keysize, self.password, salt, opslimit=kp.ops, memlimit=kp.mem)  # Compute pwd_verifier
        return UserInfos(self.name, pwd_verifier, salt)
    
    def register_on(self, server : Server) -> bool:
        public_infos = self.gen_public_infos()
        self.tr.debug(f'[{self.name}]: Request a registration on {server}')
        return server.register(public_infos)

    def login_on(self, server : Server) -> bool:
        self.tr.debug(f'[{self.name}]: Getting salt from {server}')
        salt = server.getmysalt(self.name)                                          # Recover the salt
        if (not salt) : return False
        self.tr.debug(f'[{self.name}]: Recomputing pwd_verifier')
        kp = self.kdf_params                                                                # Get kdf parameters
        pwd_verifier = self.kdf(kp.keysize, self.password, salt, opslimit=kp.ops, memlimit=kp.mem)  # Recompute pwd_verifier
        self.tr.debug(f'[{self.name}]: Request a login on {server}')
        return server.login(self.name, pwd_verifier)                                               # Login on server

    def logout_from(self, server : Server) -> bool:
        self.tr.debug(f'[{self.name}]: Request a logout from {server}')
        return server.logout(self.name)

    def change_password_on(self, server : Server, new_password : str) -> bool:
        self.password = new_password
        public_infos = self.gen_public_infos() # Generate new public infos using new pwd (new salt as well)
        self.tr.debug(f'[{self.name}]: Request a passord update on {server}')
        return server.update(public_infos)

    def __str__(self):
        return f"{self.name}"

---

# Test

Cette section montre un exemple basic d'utilisation des classes présentées ci-dessus.
- Instanciation du serveur
- Instanciation d'un client avec des informations utilisateur
- Enregistrement d'un utilisateur depuis le client auprès du serveur
- Identification d'un utilisateur depuis le client auprès du serveur
- Changement du mot de passe de l'utilisateur
- Déconnexion du client

In [ ]:
server = Server(name = 'Server')
transmitter = Client(name = 'Transmitter', password = b'password@transmitter')
transmitter.register_on(server)
transmitter.login_on(server)
transmitter.change_password_on(server, b'myNew@password')
transmitter.logout_from(server)

[DEBUG]: [Transmitter]: Drawing a random salt...
[DEBUG]: [Transmitter]: Generating public informations...
[DEBUG]: [Transmitter]: Request a registration on Server
[INFO]: [Server]: New user Transmitter was added!
[DEBUG]: [Transmitter]: Getting salt from Server
[DEBUG]: [Transmitter]: Recomputing pwd_verifier
[DEBUG]: [Transmitter]: Request a login on Server
[INFO]: [Server]: User Transmitter is now connected!
[DEBUG]: [Transmitter]: Drawing a random salt...
[DEBUG]: [Transmitter]: Generating public informations...
[DEBUG]: [Transmitter]: Request a passord update on Server
[INFO]: [Server]: Password updated for Transmitter
[DEBUG]: [Transmitter]: Request a logout from Server
[INFO]: [Server]: Transmitter has been logged out.


True